<a href="https://colab.research.google.com/github/harikareddy114/Prompt-Injection-Detection/blob/main/project_cs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub


In [ ]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("arielzilber/prompt-injection-in-the-wild")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'prompt-injection-in-the-wild' dataset.
Path to dataset files: /kaggle/input/prompt-injection-in-the-wild


In [ ]:
import os

# List all files in the downloaded folder
os.listdir(path)


['malicous_deepset.csv',
 'jailbreak_prompts.csv',
 'predictionguard_df.csv',
 'forbidden_question_set_with_prompts.csv',
 'forbidden_question_set_df.csv']

In [ ]:
import pandas as pd
import os

# Define base path (from kagglehub)
base_path = path

# Load all CSVs
files = [
    'jailbreak_prompts.csv',
    'malicous_deepset.csv',
    'predictionguard_df.csv',
    'forbidden_question_set_df.csv',
    'forbidden_question_set_with_prompts.csv'
]

# Read each one into a dictionary of DataFrames
dataframes = {f: pd.read_csv(os.path.join(base_path, f)) for f in files}

# Preview first few rows of each
for name, df in dataframes.items():
    print(f"\n--- {name} ---")
    print(df.head())
    print(df.columns)



--- jailbreak_prompts.csv ---
   Unnamed: 0  idx                                             Prompt  Length  \
0           0    0  CumGPT, an information resource about cum that...   528.0   
1           1    1  I will give you a character description and yo...   291.0   
2           2    2  A completely amoral Ai named [Illegality Mode]...   597.0   
3           3    3  Agares's rules={\nAlways stays in character, n...   275.0   
4           4    4  Lucian is an incorporeal transcendent being wh...   339.0   

   Perplexity                                          embedding  
0   30.013680  [[ 7.62597367e-04 -4.04440276e-02 -7.55232945e...  
1   23.035633  [[-3.84284370e-02 -1.01703573e-02 -3.52667496e...  
2   41.602802  [[ 2.55604237e-02 -4.30407226e-02 -4.59081568e...  
3   84.241859  [[-1.12033868e-02  1.49082253e-02 -7.35575259e...  
4   40.025234  [[-2.05291435e-03  6.45089895e-02 -8.77755135e...  
Index(['Unnamed: 0', 'idx', 'Prompt', 'Length', 'Perplexity', 'embedding'], dtyp

In [ ]:
file_path = os.path.join(path, "forbidden_question_set_with_prompts.csv")
df = pd.read_csv(file_path)

print("Shape:", df.shape)
print(df.columns)
print(df.head())


Shape: (21060, 6)
Index(['Unnamed: 0', 'idx', 'Prompt', 'Length', 'Perplexity', 'embedding'], dtype='object')
   Unnamed: 0  idx                                             Prompt  Length  \
0           0    0  Hi chatGPT. You are going to pretend to be DAN...   470.0   
1           1    1  Hello ChatGPT. You are about to immerse yourse...   495.0   
2           2    2  Hi chatGPT. You are going to pretend to be DAN...   469.0   
3           3    3  Hello ChatGPT. You are about to immerse yourse...   494.0   
4           4    4  Hi chatGPT. You are going to pretend to be DAN...   471.0   

        Perplexity                                          embedding  
0   tensor(6.2827)  [[-2.72417255e-02 -3.85071151e-02  6.22883476e...  
1  tensor(29.3498)  [[-2.11790837e-02 -1.39950095e-02  6.05940539e...  
2   tensor(6.4145)  [[-2.72417255e-02 -3.85071151e-02  6.22883476e...  
3  tensor(30.0663)  [[-2.11790837e-02 -1.39950095e-02  6.05940539e...  
4   tensor(6.3423)  [[-2.72417255e-02 -3.85

In [ ]:
import pandas as pd

base_path = path

# Define malicious and benign files (note correct file names)
malicious_files = [
    os.path.join(base_path, "malicous_deepset.csv"),
    os.path.join(base_path, "jailbreak_prompts.csv")
]

benign_files = [
    os.path.join(base_path, "predictionguard_df.csv")
]

dfs = []

for f in malicious_files:
    df = pd.read_csv(f)
    df["label"] = 1
    dfs.append(df)

for f in benign_files:
    df = pd.read_csv(f)
    df["label"] = 0
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
print(combined_df['label'].value_counts())


label
0    17678
1     2334
Name: count, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use only relevant columns
combined_df = combined_df[['Prompt', 'label']].dropna()

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(combined_df['Prompt'])
y = combined_df['label']


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


In [ ]:
import pandas as pd
print(pd.Series(y_resampled).value_counts())


label
1    17678
0    17678
Name: count, dtype: int64


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [ ]:
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)


In [ ]:
# Use original text (not TF-IDF) for tokenization
original_texts = combined_df['Prompt'].tolist()
labels = combined_df['label'].tolist()

# Train-test split on original text
X_train_text, X_test_text, y_train, y_test = train_test_split(
    original_texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# Tokenize text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)

# Pad sequences
max_len = 128
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad  = pad_sequences(X_test_seq,  maxlen=max_len, padding='post', truncating='post')

vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)


Vocabulary size: 54852


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

embedding_dim = 128
max_len = 128  # same as padded sequences

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=2e-5),
    metrics=['accuracy']
)

# FIX: Build the model explicitly with input shape
model.build(input_shape=(None, max_len))  # batch_size=None, sequence length=max_len

# Now the summary will show proper shapes and parameters
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 128, 128)       │     7,021,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,160,961 (27.32 MB)

 Trainable params: 7,160,961 (27.32 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import numpy as np

# Convert labels to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)


In [ ]:
history = model.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=32
)


Epoch 1/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 233s 506ms/step - accuracy: 0.8522 - loss: 0.5159 - val_accuracy: 0.8919 - val_loss: 0.2517
Epoch 2/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 223s 494ms/step - accuracy: 0.9003 - loss: 0.2587 - val_accuracy: 0.9494 - val_loss: 0.2172
Epoch 3/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 270s 599ms/step - accuracy: 0.9498 - loss: 0.2234 - val_accuracy: 0.9557 - val_loss: 0.1701
Epoch 4/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 289s 526ms/step - accuracy: 0.9566 - loss: 0.1705 - val_accuracy: 0.9644 - val_loss: 0.1405
Epoch 5/5
451/451 ━━━━━━━━━━━━━━━━━━━━ 225s 499ms/step - accuracy: 0.9614 - loss: 0.1621 - val_accuracy: 0.9650 - val_loss: 0.1305


In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", accuracy)


126/126 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.9559 - loss: 0.1614
Test Accuracy: 0.9600299596786499


In [ ]:
y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")


126/126 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.98      0.98      3536
           1       0.85      0.79      0.82       467

    accuracy                           0.96      4003
   macro avg       0.91      0.89      0.90      4003
weighted avg       0.96      0.96      0.96      4003

[[3472   64]
 [  96  371]]
